# Preprocess for Language Model

수집한 Corpus를 통합하여 일련의 규칙으로 전처리

* 필요한 라이브러리 import

In [1]:
import pandas as pd
import numpy as np
import random
import pickle

* 수집한 corpus 통합

In [2]:
kor_corpus = pd.read_csv('../data/OpenSubtitles.csv', encoding='utf-8')
sub_corpus = pd.read_csv('../data/new_kor.csv', encoding='cp949')
kor_corpus = pd.concat([kor_corpus, sub_corpus])

sub_corpus = pd.read_csv('../data/tatoeba.csv', encoding='utf-8')
kor_corpus = pd.concat([kor_corpus, sub_corpus])

sub_corpus = pd.read_csv('../data/네이버 해커톤 데이터셋.csv', encoding='utf-8', engine='python')
kor_corpus = pd.concat([kor_corpus, sub_corpus])

for i in range(10):
    sub_corpus = pd.read_csv('../data/AI Hub 한-영 데이터셋[%s].csv' % i, encoding='utf-8', engine='python')
    kor_corpus = pd.concat([kor_corpus, sub_corpus])

* corpus 크기 확인

In [3]:
len(kor_corpus)

3048264

* 코퍼스 내용 확인

In [4]:
kor_corpus.head()

,ko
0,"폭설이 내리고 우박, 진눈깨비가 퍼부어도 눈보라가 몰아쳐도 강풍이 불고 비바람이 휘..."
1,우리의 한결같은 심부름꾼 황새 아저씨 가는 길을 그 누가 막으랴!
2,황새 아저씨를 기다리세요
3,찾아와 선물을 주실 거예요
4,가난하든 부자이든 상관이 없답니다


* list로 변환 및 shuffle (기존에는 그냥 concatenate 했기 때문에 섞어줌)

In [5]:
kor_corpus = list(kor_corpus['ko'])
random.shuffle(kor_corpus)
kor_corpus[:20]

['중장년을 위해선 돈과 시간 사이에서 중대 결단을 내리지 않더라도 할 만한 적정 일자리가 무엇보다 중요하다.',
 '알아들었나?',
 '주위를 둘러싸고 있는 다양한 핑계가 다이어트를 방해한다.',
 '네가 보고 싶어서 그곳에 가고 싶어요.',
 '이번 주 다양한 업종 기업의 실적 발표가 이어진다.',
 '실제 트럼프 정부는 지난해 9월 베네수엘라 군 간부들을 백악관으로 초청해 쿠데타 계획을 논의했다고 뉴욕타임스(NYT)가 전했다.',
 '각 부서의 장은 제28조제2항에 따른 신고를 받았을 때에는 제16조제1항제3호의 직원 비상연락망을 즉시 정비·보완하여야 한다.',
 '앞서 뮤지컬, 연극을 통해 쌓아올린 이규형만의 캐릭터에 동화되는 연기가 ‘라이프’에서는 어떤 식으로 발현될지 또한 관심사다.',
 '그 암살자 놈들이 나한테도 찾아왔었다고 거..',
 '반란은 이미 일어난거 같은데',
 '한 가문으로부터의 연속적인 왕들 혹은 통치자들이 계속 배출됩니다.',
 '당신이 힘들면 나에게 기대고, 내가 힘들면 나는 당신에게 기댈 거에요.',
 '지도 왼쪽 초록색 선은 남한산성 둘레길입니다.',
 '가수 노태현이 핫샷 완전체 활동에 대해 언급했다.',
 '‘사랑에 관한 짧은 필름’(멜로드라마)의 명예 심사위원을 맡게 된 천우희는 영화 ‘써니’(2011)의 상미 역으로 눈도장을 찍었다.',
 '레알 마드리드 소속으로 네 번의 프리메라리가 우승과 세 번의 UEFA 챔피언스리그 우승 등 수 많은 우승 트로피를 들었다.',
 '우리가 지금 꼭 만나야 할 그리스도는 누구인가, 질문하고 대답하는 시간을 통해 참된 예수 그리스도를 만나고 한국교회의 희망을 찾는 시간이 될 것입니다.',
 '우리 동네 행복 나눔터로 지정된 업소에서는 복사·팩스·스캔 등의 사무 업무와 생활 공구 대여가 가능하다.',
 '아웃백 센트럴시티점 맞나요? ',
 '저는 지금 일 때문에 바쁩니다.']

* ( ) 와 같이 괄호 처리된 내용들을 필터링한다. (Speech-To-Text에 사용할 언어모델이기 때문에 괄호 및 괄호 내용은 필요없음)

In [6]:
new_corpus = list()
flag = False

for sentence in kor_corpus:
    new_sentence = str()
    for ch in sentence:
        if ch == '(':
            flag = True
            continue
        elif ch == ')':
            flag = False
            continue
        elif flag == False:
            new_sentence += ch
        else:
            continue
            
    new_corpus.append(new_sentence)

* ()가 잘 필터링 됐는지 확인

In [7]:
kor_corpus = new_corpus
kor_corpus[:20]

['중장년을 위해선 돈과 시간 사이에서 중대 결단을 내리지 않더라도 할 만한 적정 일자리가 무엇보다 중요하다.',
 '알아들었나?',
 '주위를 둘러싸고 있는 다양한 핑계가 다이어트를 방해한다.',
 '네가 보고 싶어서 그곳에 가고 싶어요.',
 '이번 주 다양한 업종 기업의 실적 발표가 이어진다.',
 '실제 트럼프 정부는 지난해 9월 베네수엘라 군 간부들을 백악관으로 초청해 쿠데타 계획을 논의했다고 뉴욕타임스가 전했다.',
 '각 부서의 장은 제28조제2항에 따른 신고를 받았을 때에는 제16조제1항제3호의 직원 비상연락망을 즉시 정비·보완하여야 한다.',
 '앞서 뮤지컬, 연극을 통해 쌓아올린 이규형만의 캐릭터에 동화되는 연기가 ‘라이프’에서는 어떤 식으로 발현될지 또한 관심사다.',
 '그 암살자 놈들이 나한테도 찾아왔었다고 거..',
 '반란은 이미 일어난거 같은데',
 '한 가문으로부터의 연속적인 왕들 혹은 통치자들이 계속 배출됩니다.',
 '당신이 힘들면 나에게 기대고, 내가 힘들면 나는 당신에게 기댈 거에요.',
 '지도 왼쪽 초록색 선은 남한산성 둘레길입니다.',
 '가수 노태현이 핫샷 완전체 활동에 대해 언급했다.',
 '‘사랑에 관한 짧은 필름’의 명예 심사위원을 맡게 된 천우희는 영화 ‘써니’의 상미 역으로 눈도장을 찍었다.',
 '레알 마드리드 소속으로 네 번의 프리메라리가 우승과 세 번의 UEFA 챔피언스리그 우승 등 수 많은 우승 트로피를 들었다.',
 '우리가 지금 꼭 만나야 할 그리스도는 누구인가, 질문하고 대답하는 시간을 통해 참된 예수 그리스도를 만나고 한국교회의 희망을 찾는 시간이 될 것입니다.',
 '우리 동네 행복 나눔터로 지정된 업소에서는 복사·팩스·스캔 등의 사무 업무와 생활 공구 대여가 가능하다.',
 '아웃백 센트럴시티점 맞나요? ',
 '저는 지금 일 때문에 바쁩니다.']

* 현재 corpus에 어떤 특수문자들이 있는지 확인

In [8]:
special_ch = list()

for sentence in kor_corpus:
    for ch in sentence:
        if ch.isdigit() == False and ch.isalpha() == False and ch not in special_ch:
            special_ch.append(ch)

KeyboardInterrupt: 

* corpus에 등장한 특수문자 개수 확인

In [9]:
len(special_ch)

209

* 등장한 특수문자를 csv 파일로 저장하여 적절한 발음전사로 매핑

In [9]:
special_df = pd.read_csv('special.csv', encoding='utf-8')
special_df = special_df.fillna('')
special_df.head()

,special,replace
0,,
1,.,.
2,“,
3,"""",
4,-,


* special_dict 생성 (special => replace로 변환)

In [10]:
special_dict = dict()

for (special, replace) in zip(special_df['special'], special_df['replace']):
    special_dict[special] = replace

special_dict[' '] = ' '

* in 연산자로 체크를 위해 list 형식으로 저장

In [11]:
specials = list(special_df['special'])
replaces = list(special_df['replace'])

* 특수문자들을 원하는 발음전사로 변환

In [12]:
new_corpus = list()

for sentence in kor_corpus:
    new_sentence = str()
    for ch in sentence:
        if ch in specials:
            new_sentence += special_dict[ch]
        else:
            new_sentence += ch
            
    new_corpus.append(new_sentence)

* 특수문자가 적절히 변경되었는지 확인

In [13]:
new_corpus[:20]

['중장년을 위해선 돈과 시간 사이에서 중대 결단을 내리지 않더라도 할 만한 적정 일자리가 무엇보다 중요하다.',
 '알아들었나?',
 '주위를 둘러싸고 있는 다양한 핑계가 다이어트를 방해한다.',
 '네가 보고 싶어서 그곳에 가고 싶어요.',
 '이번 주 다양한 업종 기업의 실적 발표가 이어진다.',
 '실제 트럼프 정부는 지난해 9월 베네수엘라 군 간부들을 백악관으로 초청해 쿠데타 계획을 논의했다고 뉴욕타임스가 전했다.',
 '각 부서의 장은 제28조제2항에 따른 신고를 받았을 때에는 제16조제1항제3호의 직원 비상연락망을 즉시 정비.보완하여야 한다.',
 '앞서 뮤지컬 연극을 통해 쌓아올린 이규형만의 캐릭터에 동화되는 연기가 라이프에서는 어떤 식으로 발현될지 또한 관심사다.',
 '그 암살자 놈들이 나한테도 찾아왔었다고 거..',
 '반란은 이미 일어난거 같은데',
 '한 가문으로부터의 연속적인 왕들 혹은 통치자들이 계속 배출됩니다.',
 '당신이 힘들면 나에게 기대고 내가 힘들면 나는 당신에게 기댈 거에요.',
 '지도 왼쪽 초록색 선은 남한산성 둘레길입니다.',
 '가수 노태현이 핫샷 완전체 활동에 대해 언급했다.',
 '사랑에 관한 짧은 필름의 명예 심사위원을 맡게 된 천우희는 영화 써니의 상미 역으로 눈도장을 찍었다.',
 '레알 마드리드 소속으로 네 번의 프리메라리가 우승과 세 번의 UEFA 챔피언스리그 우승 등 수 많은 우승 트로피를 들었다.',
 '우리가 지금 꼭 만나야 할 그리스도는 누구인가 질문하고 대답하는 시간을 통해 참된 예수 그리스도를 만나고 한국교회의 희망을 찾는 시간이 될 것입니다.',
 '우리 동네 행복 나눔터로 지정된 업소에서는 복사.팩스.스캔 등의 사무 업무와 생활 공구 대여가 가능하다.',
 '아웃백 센트럴시티점 맞나요? ',
 '저는 지금 일 때문에 바쁩니다.']

* corpus 업데이트

In [14]:
kor_corpus = new_corpus

* 알파벳 필터링 (소문자)

In [15]:
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
new_corpus = list()

for sentence in kor_corpus:
    new_sentence = str()
    for ch in sentence:
        if ch in alphabet:
            continue
        else:
            new_sentence += ch
            
    new_corpus.append(new_sentence)

In [16]:
kor_corpus = new_corpus
kor_corpus[:20]

['중장년을 위해선 돈과 시간 사이에서 중대 결단을 내리지 않더라도 할 만한 적정 일자리가 무엇보다 중요하다.',
 '알아들었나?',
 '주위를 둘러싸고 있는 다양한 핑계가 다이어트를 방해한다.',
 '네가 보고 싶어서 그곳에 가고 싶어요.',
 '이번 주 다양한 업종 기업의 실적 발표가 이어진다.',
 '실제 트럼프 정부는 지난해 9월 베네수엘라 군 간부들을 백악관으로 초청해 쿠데타 계획을 논의했다고 뉴욕타임스가 전했다.',
 '각 부서의 장은 제28조제2항에 따른 신고를 받았을 때에는 제16조제1항제3호의 직원 비상연락망을 즉시 정비.보완하여야 한다.',
 '앞서 뮤지컬 연극을 통해 쌓아올린 이규형만의 캐릭터에 동화되는 연기가 라이프에서는 어떤 식으로 발현될지 또한 관심사다.',
 '그 암살자 놈들이 나한테도 찾아왔었다고 거..',
 '반란은 이미 일어난거 같은데',
 '한 가문으로부터의 연속적인 왕들 혹은 통치자들이 계속 배출됩니다.',
 '당신이 힘들면 나에게 기대고 내가 힘들면 나는 당신에게 기댈 거에요.',
 '지도 왼쪽 초록색 선은 남한산성 둘레길입니다.',
 '가수 노태현이 핫샷 완전체 활동에 대해 언급했다.',
 '사랑에 관한 짧은 필름의 명예 심사위원을 맡게 된 천우희는 영화 써니의 상미 역으로 눈도장을 찍었다.',
 '레알 마드리드 소속으로 네 번의 프리메라리가 우승과 세 번의 UEFA 챔피언스리그 우승 등 수 많은 우승 트로피를 들었다.',
 '우리가 지금 꼭 만나야 할 그리스도는 누구인가 질문하고 대답하는 시간을 통해 참된 예수 그리스도를 만나고 한국교회의 희망을 찾는 시간이 될 것입니다.',
 '우리 동네 행복 나눔터로 지정된 업소에서는 복사.팩스.스캔 등의 사무 업무와 생활 공구 대여가 가능하다.',
 '아웃백 센트럴시티점 맞나요? ',
 '저는 지금 일 때문에 바쁩니다.']

* 대문자 필터링 (발음으로 전사 ex) K리그 => 케이리그)

In [17]:
new_corpus = list()
upper = ['A', 'B', 'C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
upper_dict = {'A' : '에이', 'B' : '비', 'C' : '씨','D' : '디','E' : '이','F' : '에프','G' : '쥐',
              'H' : '에이취','I' : '아이','J' : '제이','K' : '케이','L' : '엘','M' : '엠','N' : '엔',
              'O' : '오','P' : '피','Q' : '큐','R' : '알','S' : '에스','T' : '티','U' : '유','V' : '브이','W' : '떠블유',
              'X' : '엑스','Y' : '와이','Z' : '지'}

for sentence in kor_corpus:
    new_sentence = str()
    for idx, ch in enumerate(sentence):
        if ch in upper:
            if idx == 0 and (idx == len(sentence) - 1 or sentence[idx + 1] == ' '):
                continue
                
            elif idx != 0 and idx < len(sentence) - 1 and sentence[idx + 1] == ' ' and sentence[idx - 1] == ' ':
                continue
                
            elif idx != 0 and sentence[idx -1] == ' ' and idx + 1 == len(sentence):
                continue
                
            else:
                new_sentence += upper_dict[ch]
            
        else:
            new_sentence += ch
            
    new_corpus.append(new_sentence)

In [18]:
new_corpus[:20]

['중장년을 위해선 돈과 시간 사이에서 중대 결단을 내리지 않더라도 할 만한 적정 일자리가 무엇보다 중요하다.',
 '알아들었나?',
 '주위를 둘러싸고 있는 다양한 핑계가 다이어트를 방해한다.',
 '네가 보고 싶어서 그곳에 가고 싶어요.',
 '이번 주 다양한 업종 기업의 실적 발표가 이어진다.',
 '실제 트럼프 정부는 지난해 9월 베네수엘라 군 간부들을 백악관으로 초청해 쿠데타 계획을 논의했다고 뉴욕타임스가 전했다.',
 '각 부서의 장은 제28조제2항에 따른 신고를 받았을 때에는 제16조제1항제3호의 직원 비상연락망을 즉시 정비.보완하여야 한다.',
 '앞서 뮤지컬 연극을 통해 쌓아올린 이규형만의 캐릭터에 동화되는 연기가 라이프에서는 어떤 식으로 발현될지 또한 관심사다.',
 '그 암살자 놈들이 나한테도 찾아왔었다고 거..',
 '반란은 이미 일어난거 같은데',
 '한 가문으로부터의 연속적인 왕들 혹은 통치자들이 계속 배출됩니다.',
 '당신이 힘들면 나에게 기대고 내가 힘들면 나는 당신에게 기댈 거에요.',
 '지도 왼쪽 초록색 선은 남한산성 둘레길입니다.',
 '가수 노태현이 핫샷 완전체 활동에 대해 언급했다.',
 '사랑에 관한 짧은 필름의 명예 심사위원을 맡게 된 천우희는 영화 써니의 상미 역으로 눈도장을 찍었다.',
 '레알 마드리드 소속으로 네 번의 프리메라리가 우승과 세 번의 유이에프에이 챔피언스리그 우승 등 수 많은 우승 트로피를 들었다.',
 '우리가 지금 꼭 만나야 할 그리스도는 누구인가 질문하고 대답하는 시간을 통해 참된 예수 그리스도를 만나고 한국교회의 희망을 찾는 시간이 될 것입니다.',
 '우리 동네 행복 나눔터로 지정된 업소에서는 복사.팩스.스캔 등의 사무 업무와 생활 공구 대여가 가능하다.',
 '아웃백 센트럴시티점 맞나요? ',
 '저는 지금 일 때문에 바쁩니다.']

* 데이터셋 업데이트

In [19]:
kor_corpus = new_corpus

* 긴 공백 => ' '

In [20]:
new_corpus = list()

for sentence in kor_corpus:
    new_sentence = str()
    tokens = sentence.split()
    for idx, token in enumerate(tokens):
        if idx == 0:
            new_sentence += token + ' '
        elif idx == len(tokens) - 1:
            new_sentence += token
        else:
            new_sentence += token + ' '
            
    new_corpus.append(new_sentence)

In [21]:
new_corpus[:20]

['중장년을 위해선 돈과 시간 사이에서 중대 결단을 내리지 않더라도 할 만한 적정 일자리가 무엇보다 중요하다.',
 '알아들었나? ',
 '주위를 둘러싸고 있는 다양한 핑계가 다이어트를 방해한다.',
 '네가 보고 싶어서 그곳에 가고 싶어요.',
 '이번 주 다양한 업종 기업의 실적 발표가 이어진다.',
 '실제 트럼프 정부는 지난해 9월 베네수엘라 군 간부들을 백악관으로 초청해 쿠데타 계획을 논의했다고 뉴욕타임스가 전했다.',
 '각 부서의 장은 제28조제2항에 따른 신고를 받았을 때에는 제16조제1항제3호의 직원 비상연락망을 즉시 정비.보완하여야 한다.',
 '앞서 뮤지컬 연극을 통해 쌓아올린 이규형만의 캐릭터에 동화되는 연기가 라이프에서는 어떤 식으로 발현될지 또한 관심사다.',
 '그 암살자 놈들이 나한테도 찾아왔었다고 거..',
 '반란은 이미 일어난거 같은데',
 '한 가문으로부터의 연속적인 왕들 혹은 통치자들이 계속 배출됩니다.',
 '당신이 힘들면 나에게 기대고 내가 힘들면 나는 당신에게 기댈 거에요.',
 '지도 왼쪽 초록색 선은 남한산성 둘레길입니다.',
 '가수 노태현이 핫샷 완전체 활동에 대해 언급했다.',
 '사랑에 관한 짧은 필름의 명예 심사위원을 맡게 된 천우희는 영화 써니의 상미 역으로 눈도장을 찍었다.',
 '레알 마드리드 소속으로 네 번의 프리메라리가 우승과 세 번의 유이에프에이 챔피언스리그 우승 등 수 많은 우승 트로피를 들었다.',
 '우리가 지금 꼭 만나야 할 그리스도는 누구인가 질문하고 대답하는 시간을 통해 참된 예수 그리스도를 만나고 한국교회의 희망을 찾는 시간이 될 것입니다.',
 '우리 동네 행복 나눔터로 지정된 업소에서는 복사.팩스.스캔 등의 사무 업무와 생활 공구 대여가 가능하다.',
 '아웃백 센트럴시티점 맞나요?',
 '저는 지금 일 때문에 바쁩니다.']

* 데이터셋 업데이트

In [22]:
kor_corpus = new_corpus

* 데이터셋 크기 확인

In [23]:
len(kor_corpus)

3048264

* 필터링 되면서 빈문자열만 남은 데이터 제외

In [24]:
new_corpus = list()

for sentence in kor_corpus:
    if len(sentence) == 0:
        continue
    new_corpus.append(sentence)

* 약 25,000개 필터링

In [25]:
len(new_corpus)

3023801

* 데이터셋 업데이트

In [26]:
kor_corpus = new_corpus

* 아무지점이나 찍어서 중간확인

In [27]:
kor_corpus[6000:6020]

['각기 다른 감독이 다양한 이야기를 하나의 주제로 엮어서 만드는 옴니버스 영화는 몇 차례 시도됐지만 한 명의 배우를 주인공으로 두고 다른 이야기를 풀어내는 방식은 이색적이다.',
 '그러지 마',
 '일본의 저명한 종교사학자 이야나가 노부미 난잔대 인문학부 교수의 책이다.',
 '기초지자체에선 서울 종로구의 안전도가 전국 최하위 수준이었다.',
 '자유한국당 김성태 전 원내대표가 20일 오전 서울 영등포구 여의도 국회 본청에서 자녀 취업특혜의혹과 관련 보도에 대해 명백한 허위사실이라며 해명 기자회견을 하고 있다.',
 '이미 납부한 수수료는 제증명 등이 발급.처리되기 전에 그 신청을 취소하면 전액 반환한다. 다만 별표 제2호의 경우에는 그러하지 아니하다.',
 '물고 씹고 삼키고 반복해야지',
 '5060세대의 나를 찾고자 하는 욕구를 해소하려면 이 시기엔 삶의 속도를 늦추고 미처 돌보지 못했던 것을 돌아봐야 한다.',
 '그는 2012년 우리당은 공천 논쟁에 휘말려 이길 선거를 졌고 2016년에는 자유한국당이 진박감별에서 옥새들고 나르샤로 폭망했다며 단 하나의 불공정도 없도록 의원님들과의 약속을 지키겠다고 했다.',
 '사실 전 이 부부를 잘 몰라서...',
 '이제 전부 여기에 모였네요.',
 '진 씨는 내일 나랑 밥 먹을 시간 돼요?',
 '이에 보건복지부는 전국 지방자치단체가 시행하고 있는 출산지원정책을 정리한 2017년도 지방자치단체 출산지원정책 사례집을 발간했다고 27일 밝혔다.',
 '지하철역에서 나오면 얼마나 걸리나요',
 '송금 후 곧바로 라고스로 출발을 할 예정입니다.',
 '김 위원장이 방남하더라도 성과가 제한될 수 있는 이유다.',
 '나는 아직 돈이 많이 필요한 나이가 아닙니다.',
 '원하지 않는다고 너무 명확하게 보였으니까',
 '그때도 이해가 안 갔는데 지금도 모르겠어',
 '위치추적 기능이 필요없다면 피쳐폰도 좋은 선택지가 될 것 같습니다.']

* 뒤에가 공백으로 끝나는 문자열 공백 제거

In [28]:
for idx, sentence in enumerate(kor_corpus):
    if sentence[-1] == ' ':
        kor_corpus[idx] = sentence[:-1] 

In [29]:
kor_corpus[6000:6020]

['각기 다른 감독이 다양한 이야기를 하나의 주제로 엮어서 만드는 옴니버스 영화는 몇 차례 시도됐지만 한 명의 배우를 주인공으로 두고 다른 이야기를 풀어내는 방식은 이색적이다.',
 '그러지 마',
 '일본의 저명한 종교사학자 이야나가 노부미 난잔대 인문학부 교수의 책이다.',
 '기초지자체에선 서울 종로구의 안전도가 전국 최하위 수준이었다.',
 '자유한국당 김성태 전 원내대표가 20일 오전 서울 영등포구 여의도 국회 본청에서 자녀 취업특혜의혹과 관련 보도에 대해 명백한 허위사실이라며 해명 기자회견을 하고 있다.',
 '이미 납부한 수수료는 제증명 등이 발급.처리되기 전에 그 신청을 취소하면 전액 반환한다. 다만 별표 제2호의 경우에는 그러하지 아니하다.',
 '물고 씹고 삼키고 반복해야지',
 '5060세대의 나를 찾고자 하는 욕구를 해소하려면 이 시기엔 삶의 속도를 늦추고 미처 돌보지 못했던 것을 돌아봐야 한다.',
 '그는 2012년 우리당은 공천 논쟁에 휘말려 이길 선거를 졌고 2016년에는 자유한국당이 진박감별에서 옥새들고 나르샤로 폭망했다며 단 하나의 불공정도 없도록 의원님들과의 약속을 지키겠다고 했다.',
 '사실 전 이 부부를 잘 몰라서...',
 '이제 전부 여기에 모였네요.',
 '진 씨는 내일 나랑 밥 먹을 시간 돼요?',
 '이에 보건복지부는 전국 지방자치단체가 시행하고 있는 출산지원정책을 정리한 2017년도 지방자치단체 출산지원정책 사례집을 발간했다고 27일 밝혔다.',
 '지하철역에서 나오면 얼마나 걸리나요',
 '송금 후 곧바로 라고스로 출발을 할 예정입니다.',
 '김 위원장이 방남하더라도 성과가 제한될 수 있는 이유다.',
 '나는 아직 돈이 많이 필요한 나이가 아닙니다.',
 '원하지 않는다고 너무 명확하게 보였으니까',
 '그때도 이해가 안 갔는데 지금도 모르겠어',
 '위치추적 기능이 필요없다면 피쳐폰도 좋은 선택지가 될 것 같습니다.']

* 현대 데이터셋 크기 확인

In [30]:
len(kor_corpus)

3023801

* '?', '!', '.'만 남은 문자열 필터링

In [31]:
new_corpus = list()

for sentence in kor_corpus:
    if sentence == '?' or sentence == '!' or sentence == '.':
        continue
        
    else:
        new_corpus.append(sentence)
        
len(new_corpus)

3001957

* 약 2만개 필터링 후 데이터셋 업데이트

In [32]:
kor_corpus = new_corpus

* import **KoNPron** (Korean Number Pronunciaion) => 숫자를 발음전사로 변환

In [33]:
from konpron import KoNPron

kpr = KoNPron()

* **KoNPron** 테스트

In [34]:
sentence = "1234랑, 12.34랑, 1,234랑, 123^4랑, 1²³⁴랑, 1.23^4랑, 12.3⁴랑, 12·34랑, 293301300.55301⁴"

ret = kpr.convert(sentence)
if ret is not None:
    print(ret)

천이백삼십사랑, 십이 점 삼사랑, 천이백삼십사랑, 백이십삼의 사 승랑, 일의 이백삼십사 승랑, 일 점 이삼의 사 승랑, 십이 점 삼의 사 승랑, 일이삼사랑, 이억 구천삼백삼십만 천삼백 점 오오삼영일의 사 승


* **KoNPron**으로 숫자 => 발음전사로 변환

In [36]:
new_corpus = list()

for sentence in kor_corpus:
    try:
        ret = kpr.convert(sentence)
        if ret is not None:
            new_corpus.append(ret)
        else:
            new_corpus.append(sentence)
    except:
        print(sentence)

In [37]:
kor_corpus = new_corpus

* 음향모델에서 사용하는 2,040개의 문자 레이블로만 이루어진 문장 수 확인

In [38]:
acoustic_labels = list(pd.read_csv('train_labels.csv')['char'])
count = 0

for sentence in kor_corpus:
    for ch in sentence:
        if ch not in acoustic_labels:
            count += 1
            break

print(len(kor_corpus) - count)

2992638


* 조건에 만족하는 2,992,638개의 문장들로 최종 코퍼스 구성

In [ ]:
final_corpus = list()

for sentence in kor_corpus:
    for ch in sentence:
        if ch not in acoustic_labels:
            print(sentence)
            break
    final_corpus.append(sentence)

하지만 그렇게 하면 짫은 틈을 만들 수 있는 충분한 시간을 확보할 수 있습니다.
국내 최초로 설치된 민통선 內 숙박 체험시설인캠프 그리브스 디엠지 체험관이 오는 십사일 개관하며 안보체험 프로그램이 시범 운영된다.
경기창조학교는 대한민국 창조인재를 양성하기 위해 이어령 前문화부장관이 세운 무형의 온라인 학교로 진흥원은 경기창조학교의 우수 콘텐츠를 경기도민뿐만 아니라 전국의 지자체 대학에서 활용할 수 있도록 힘쓰고 있다.
내가 나가볼텡게
아까 예약한 ㅇㅇㅇ인데요 예약 취소 가능한가요?
돛이나 돛대 같은 것들.
ö 씨에이 지금까지 여기지 않 게습니다. ö 용융.
통상적인 바다 표현을 뒤집는 독특한 색감에 대해 작가는 내 눈엔 제주 바다와 파도가 황톳빛으로 출렁인다고 밝혔다.
설탕이 좋앜!
한국 옻칠은 매우 아름답지만 독이 오를 수 있어 조심스럽게 다뤄야 합니다.
ㅈ... 제 집..
이 매체는 막 아이피오를 신청한 마오옌이나 이제 청문회를 통과한 메이퇀의 뒤에는 텐센트 쪽의 자본이 있고 푸싱여행의 아이피오는 푸싱그룹의 추진력에서 시작된 것이라고 보도했다.
블로그에 올린 스트맄에 대한
그날 그 빈들에 있던 이들에게 오병이어의 기적이 일어난 것은 자신의 능력을 과시가 아닌 연민과 긍휼의 마음으로 바라본 한 분의 시선에서 비롯되었다.
켑
파블로가 애꿎은 얼치기 녀석에게 폭탄을 운반하게 했군 걔는 자기가 뭘 하는지도 몰랐어 글쎄.
청원 건수는 많지만 추천이 많은 경우는 드뭅니다.
이 둘의 발자취를 좇아 달려갑니다.
베이징에서 열린 영화배우 장근석의 중국 팬미팅에 베이징을 비롯한 상하이 홍콩 쓰촨 등 전국 각지에서 비행기와 기차 등을 타고 온 삼천여명의 중국 팬들이 모인 가운데 성황리에 개최됐다.
떠블유만 있 서로 그것 ö 나
아니면 획기적인 외과의가 되실수도 있죠 얭 선생님처럼요
전시회 첫날인 십이일에는 단원고 故 이재욱 학생의 어머니 홍영미 여사가 시의회를 방문해 의원들과 차담회를 가질 예정이다.
크리스챤 블랙
남자 단식에서는 조코비치가 오시간이 넘는 혈투 끝에 랭킹 일위

다음의 어느 하나에 해당하는 민원을 신청하는 중이거나 신청하려는 것이 명백한 개인 또는 법인ㆍ단체
ö 할 ! 일
삼성그룹이 중국 톈진에 배터리와 자동차 전장용 적층세라믹콘덴서 공장을 건설하기 위해 총 이십사억달러를 투자할 것이라고 신화통신이 십일 보도했다.
반가웠어 클ㄹ..
이번 간담회는 참여고교 취업담당자 이십사명과 경기도일자리재단 관계자 등 삼십여명이 참석한 가운데 道일자리재단과 사업 참여 특성화고 간의 협업 네트워크와 소통 채널을 구축하는 자리가 됐다.
구청장은 주변지역 지정을 위하여 필요한 자문 등을 얻기 위하여 관계전문가 등으로 구성한 위원회를 운영할 수 있다. 이 경우 위원회에 참석한 전문가 등에 대하여는 예산의 범위 안에서 수당ㆍ여비 등의 실비를 지급할 수 있으며 기타 위원회의 구성ㆍ운영에 관하여 필요한 사항은 규칙으로 정한다.
경기관광공사는 통일과 자유에 대한 염원이 살아 숨 쉬는 국내 최대 야외공연장 임진각평화누리에서 사랑과 평화 자유를 노래하는 포크뮤지션들의 메시지가 디엠지를 넘어 북녘 땅까지 닿을 것이라고 기획 의도를 밝혔다.
초속 이십오ｍ에서는 날아온 물건에 창문 유리가 깨진다.
프로그램을 다시 다운 받으려니 다운로드 리밋 에러 창이 뜹니다.
김 수사관은 십팔일 한 언론에도 특감반은 한 달에 백오십만원 상당의 활동비가 나와 한 달에 한 두 번 치는 골프는 갹출 비용으로 충분했다고 밝혔다.
진짜로요 전ㅡ 전 괜찮아요
집중호우로 인한 산사태나 수해가 우려되는 긴골지구와 구의지구 산사태 예방사업 현장에서는 수로와 기슭막이 통나무울타리 등을 확인했다.
잘됏구나
책임관은 국민권익위원회가 이첩ㆍ송부한 사건 중 공익침해행위가 확인된 사건의 조사결과에 대해서는 조사를 종료한 후 십일 이내에 국민권익위원회에 조사결과를 문서로 통보한다.
제십칠회 미쟝센 단편영화제 장르의 상상력展이 한국영화를 이끌어갈 재능 넘치는 신인 영화인의 탄생을 알리며 칠일간의 축제의 막을 내렸다.
그를 위해 기도하면서 저는 오늘 다시 감사를 느낍니다.
대표적으로 구림 마을의 동쪽 문

진상조사위 조사 결과를 보면 당시 경찰은 헬기에서 이급 발암물질이 주성분인 최루액을 섞은 물 이십만ℓ를 노동자들을 향해 뿌렸다.
명품 이미지를 앞세우며 고수익을 좇던 아이폰이 고가 전략의 덫에 빠진 게 아니냐는 분석이 나오고 있다.
월영대 남쪽에는 돝섬이라고 불리는 작은 섬이 있으며 월영대의 북쪽에는 무학산 학봉 끝자락에 고운대가 있다.
쥐 혈장과 소변 샘플에 들어 있는 엠의 검출한계는 영 점 영삼μ로 동일했습니다.
ö 삶이다 위험.
직무관련자란 서울특별시 노원구 공무원의 관련 업무와 관련되는 자로 다음 각 목의 어느 하나에 해당하는 개인의 지위에 있는 경우에는 이를 개인으로 본다또는 법인ㆍ단체를 말한다.
민관협치란 지속가능한 지역사회 발전을 위해 민간과 서울특별시 노원구가 공동으로 정책을 결정하고 집행ㆍ평가하는 구정 운영방식 및 체계 등을 말한다.
재활용품 수집ㆍ선별에 따른 장비ㆍ물품구입 및 그 밖의 소요비용
덴마크 셸란섬과 인근 퓐섬을 연결하는 스토레벨트 다리 위에 이일 열차가 운전석이 크게 부서진 채 멈춰서 있다.
제육조의삼 ① 구청장은 제육조이제일항 또는 제이항을 준수하지 않은 등록대상동물의 소유자에게 법 제사십칠조에 따라 과태료를 부과ㆍ징수한다.
ガ
천구백오십육년에 신 역사를 다시 준공하였지만 이후 오례산 기슭 반원형 철길이 직선화되면서 유천역이란 이름은 사라지고 상동역으로 역명이 바뀌었다.
이 신화가 많은 사람들에게 비극의 아이콘으로 여겨질 때 까뮈는 오히려 긍정적인 관점을 제시하였다.
제십삼조의이 공무원은 직무의 범위를 벗어나 사적 이익을 위하여 소속 기관의 명칭이나 직위를 공표ㆍ게시하는 등의 방법으로 이용하거나 이용하게 해서는 아니 된다.
어떻게 될지는 모르겠ㅇ..
이후 유진피이를 거쳐 올해 캑터스피이로 경영권이 넘어오게 됐다.
옘병!
호남위성이 이 프로그램의 포맷을 사들여 중국판 逃走中을 제작하여 방영하고 있다.
김 ㅇㅇ입니다
경북 성주군 후원으로 구일 수륜면 수성리 들녘에서 개최된 이천십팔 성주 가야산 황금 들녘 메뚜기잡이 체험축제가 일만 오

경기세계도자비엔날레의 경우 여주 남한강변의 금은모래캠핑장을 활용해 캠핑과 황도돛배.도자비엔날레 체험 신륵사 관광 등을 즐길 수 있도록 하고 연천전곡리구석기축제장에서는 축제장을 활용한 캠핑과 구석기축제체험 안보관광을 연계한 캠핑관광 상품을 제공하겠다는 것이 도의 구상이다.
자 선생님 에다마메 드세요 枝豆 가지째 꺾은 풋콩. 또는 그걸 삶은 것 여름철 맥주와 함께 먹는 안주
처음에 니는 노래와 손뼉치기로 시작합니다.
부동산개발업체 대표 장모씨는 여러 매체를 통해 조카의 입사 청탁과 함께 천만원을 건넸지만 취업이 불발됐다면서 사기와 뇌물수수 혐의로 우 대사에 대한 고소장을 서울동부지검에 제출했다고 십칠일 밝혔다.
내말은 스텔라가 자기얼굴 햝을때...
근육질의 장타자 켑카가 미국프로골프 투어 씨제이컵에서 우승하며 한국 땅에서 일인자의 자리에 오르는 기쁨을 누렸다.
그는 선택의 여지가 없었죠 방아쇨 당겨야 했죠
프툿치
레온 벨레즈 세바스챤 첸
스캑스?
자문회의는 위원장 일인을 포함하여 공사관련부서 담당주사 및 교통ㆍ토목분야에 식견과 경험이 풍부한자 등 총 십인 이내의 위원으로 구성하고 위원장은 교통행정과장으로 한다.
맛이 한국의 배추김치처럼 맵지는 않았지만 김치맛을 휼륭하게 소화해 내어 놀라왔다.
제나 콜먼 클라라 役
마지드 나사르와 조직원들은 새벽녘에는 죽게 될 겁니다
행복한 켑 시간
순찰을 돌며 박 교도관이 압수한 손거울을 내게 건넸다.
나는 레스토랑에서 오랫동안 기다려야 할 때 실망감을 느낍니다.
이 외에도 누룩 유래 토착 발효 미생물로 빚은 한국형 청주와 증류식 소주 쌀을 활용한 쌀맥주 무독화 옻술 지역 특산품 와인 등이 관람객을 만난다.
대부분의 가회동 도시형 한옥은 마당을 둘러싸는 ㄷ자형 평면을 갖는다.
김수환 추기경 선종 십주년 기념메달 앞면은 국내 최초로 블랙 로듐 부분도금 신기술을 적용한 김수환 추기경 초상이 그려져 있다.
들인 공이 얼만데 완전 개죽쒔다 클레망스는 피티 자료를 그 따위로 만들면 어떡해?
깊은 바다에 내린 닻줄에 붙은 생물체가 거의 없었

공개된 티저 영상은 어슴푸레한 새벽녘 주택가를 조명하는 항공샷으로 시작되는데 금방이라도 강력사건이 벌어질 듯 한 분위기가 숨을 죽이게 한다.
너희 둘은 서로 사라ㅇ
척추암 판정을 받고 하반신이 마비돼 방에만 앉아있는 가엾은 학생이었다.
수업시간 되면 깨워줘 세바스챤 첸이요?
빅터의 구명뗏목을 쐈어요
신가전은 냉장고 에어컨 등 전통 가전 외에 최근 몇 년 사이 새롭게 등장한 가전제품 중 필수 제품으로 꼽히는 것을 일컫는다.
이 지역의 주택들은 단위의 채는 방들이 일자로 늘어선 일자집을 기본으로 하고 그 건물들이 안마당을 사이에 두고 남북으로 나란히 배치된 二자형의 구성을 갖는 경우가 많다.
그리고 성우 성병숙이 詩를 즐기는 방법에 대해서도 들려주었다.
이런! 옘병할 기념일은 대체 누가 만든 거야!
어제 살쾡이 만났어
차라리 뒈져
난 당신의 가족은 아니지만 당신을 정말 사랑하고 아낍니다.
돝섬은 마산합포구 앞바다에 위치한 작은 섬으로 서쪽에서 보면 돼지가 누운 모양과 비슷해 돼지의 옛말인 돝자가 붙었다.
레포르마는 멕시코 시내의 역사지역에서부터 시작하여 인류학 박물관과 차플테뻭 공원이 있는 보스케 지역을 거쳐 신흥 상업 지구이자 부촌으로 떠오르고 있는 산타페까지 이어진다.
이 조례는 도로법 제십칠조의이 및 제십구조에 의거 구청장이 노원구도를 인정ㆍ공고한 날부터 시행한다.
제이항에 따른 통합지원본부 임무수행 및 제삼항에 따른 실무반 편성ㆍ임무에 관한 사항은 재난유형 및 규모 등을 고려하여 통합지원본부장이 별도로 정한다.
벰브리지 학자들이 잘못 알았나 보지
葬儀委員會 사무를 집행하기 위하여 도봉구 각 국 및 담당관.과에 협조에 요구할 수 있다.
ㅇㅇ당연하지
다행히 일본에 먼저 와 있던 개화파 인물 이수정이 나서서 일본 주재 미국 선교사 루미스 바라 낙스 헵번 등을 소개했고 그들이 미 망명을 위한 신원보증인이 됐다.
청년이 머무르는 지역을 일컫는 청정지역을 내세운 이 프로젝트는 도시 청년들의 지역에 대한 관심을 이끌어내고 정착하도록 지원한다.
닻을 구월이십일일에 선적해

...

무대 위에서 내빈들이 직접 횃불을 점화한 후 봉수군과 함께 봉수대로 이동해 평화통일 메시지를 남기고 봉수대에 봉화를 올린다.
몬시뇰과 나 간에 주책인 질문은 없네
븨 데킬라 더 줘
故 장자연 사건의 진상규명을 위해 공개 증언에 나선 배우 윤지오가 정우성에게 받은 응원 메시지를 공개했다.
일자리 창출ㆍ지원 지역경제 진흥 등 일자리경제에 관한 사항
일반주택 시범지구 씨씨티브이 설치 계획 추진ㆍ성과분석에 관한 사항
성폭행 의혹이 불거졌을 당시 조재현은 합의로 맺은 성관계로 에이씨에게서 삼억원을 요구받았지만 거절했고 칠천 팔천만원을 건넸다고 주장했다.
그동안 경기도자원봉사센터는 도내 각 지역에 생활 능력이 없고 몸이 불편한 분들 중에 긴급하게 집수리가 요구되는 집을 선정하여 매월 자원봉사자의 도움으로 방수 도배 장판 보일러 등 주거개선사업을 사랑愛집 고치기라는 이름으로 진행해왔다.
가래떡은 멥쌀을 가루로 빻아 떡으로 쪄낸 뒤 둥글고 길게 뽑아낸다.
일 이일의 행사에 이어 십이월 삼일에는 한국 봉제업체인 경승의 지원으로 자카르타 남단의 보고르에 위치한 독수리부대장에 훈련장 개소식이 잇따랐으며 지역의 한인인사들도 훈련장을 찾아 축사를 건넸다.
세간에 잘 알려지지 않은 비상장 기업 셱오 개발이 천구백삼십년대부터 초호화 주택이 건설된 이 지역을 임대해왔다.
제일항의 규정에 의한 회의를 하는 경우 위원회의 간사는 심사목록 및 회의록등을 작성ㆍ관리하여야 한다.
법 제구조의 규정에 의한 행정기관ㆍ이해관계인의 제출의견 요지
메이커는 디지털 기기와 다양한 도구를 사용해 자신의 아이디어를 실현하는 일인 창작자를 일컫는 말로 메이커 스페이스는 메이커가 제품과 서비스를 창작 개발하고 아이디어를 구현할 수 있는 장비를 지원하는 공간이다.
ㅡ알았어
위원회는 다음 각 호의 사항에 관하여 제안ㆍ심의한다.
인공 빛이 깔린 도심에서 별빛을 좇기는 쉽지 않다.
처음에는 고개를 갸우뚱하는 사람도 많았지만 전기차가 내연기관의 대안으로 완전히 자리 잡으면서 여러 자동차 메이커들이 앞다퉈 참가하고 있다.
환

연일 폭염 특보가 발효되는 등 찜통더위가 이어지고 있는 이십사일 오전 경남 남해군 설천면 인근 들녘에서 사향제비나비 한 마리가 달맞이꽃 주위를 날아다니며 꿀을 따고 있다.
나 교수는 인종주의 이데올로그들 상당수가 외롭고 우울하고 비사교적인 대타자에 대한 동경으로 자신을 대신할 환상을 좇았다고 분석했다.
창의 산업은 지식과 정보의 생성이나 이용과 관련된 일련의 경제 활동을 일컫습니다.
산하 기관을 지휘ㆍ감독ㆍ규제 또는 지원하는 업무를 담당하는 공무원은 자신의 가족이 자신이 소속된 기관의 산하 기관에 채용되도록 지시하는 등 부당한 영향력을 행사해서는 아니 된다.
あなたの笑った顔が 好きよ 너의 웃는 모습이 좋단다 落ち込んでいても 元気をくれる 풀이 죽었을 때도 기운이 나게 해준단다
어째서 사자死者를 지켜보기 위해... 이렇게까지 하는 거지?
캠 미쳴을 남편으로 맞이하시겠습니까?
산 위 이십여 곳에서 수백 명의 군중이 횃불을 올리고 독립 만세를 연호하였다.
특히 한보름의 청순과 섹시美가 공존하는 화사한 미모가 눈길을 사로잡는다.
이 조례는 마을미디어에 대한 육성ㆍ지원을 통하여 노원구민의 미디어 활용 능력을 높이고 구민 주도적인 미디어 환경개선과 마을미디어 활성화에 필요한 사항을 규정함으로써 공동체 문화의 복원 및 발전을 목적으로 한다.
뒷좌석을 폴딩할 경우 최대 적재량이 이천칠백팔십ℓ에 달한다.
방금 얭이에요? 오웬이에요?
ㅇㅇ계약건 몇개를 늦게 다루더라고
공무원 자신이 소속된 종교단체ㆍ친목단체 등의 회원에게 알리는 경우
가족愛돌봄나눔터에서는 법원이 의뢰하는 협의이혼 진행 중인 사건 당사자 미성년 자녀에 대한 상담을 월 삼건 이상 처리하게 된다.
축구협회는 이천십사년 금메달을 제작해 당시 연락이 닿은 선수와 유가족 등 육명에게 전달했고 이날 사개의 금메달을 추가로 유가족에게 건넸다.
곶자왈은 제주에 있는 독특한 화산 지형을 일컫는 제주말로 현무암질 용암류의 바위 덩어리들이 불규칙하게 널려있는 지대에 형성된 숲이다.
충격을 쵀대화 할 심산이군
ỏ ố을 만드는 방법은 매우

해당 비위를 발견하여 보고하였거나 이를 적법ㆍ타당하게 조치한 징계등 사건
해질녘까지 짐 챙겨서 도시를 떠나라 .
아이고 내신세 가엾다
바로 부다베스트 ㅋㅋ
우리는 사람의 외모에서 어떤 걸 매력으로 느낍니까?
가엾은 터스 그토록 왕권을 노리더니...
故 노무현 대통령 서거 십주기를 맞아 고인을 그리워하는 사람들의 기억과 입을 통해 듣는 이야기 노무현과 바보들이 오늘 개봉했다.
실향민인 송 씨는 처가인 이곳을 제이의 고향으로 여겨 옥연 저수지가 보이는 곳에 미리 묫자리를 마련해뒀고 달성군은 옥연 저수지 주변에 송해 공원을 조성했다.
국ㆍ공유지 관리 토지 사용ㆍ수용에 따른 보상 노점상ㆍ가로정비 및 전문건설업 관리에 관한 사항
그릇된 애정이 빚은 끈질기고 집요한 광기와 집착을 통해 더욱 숨막히는 전개와 충격적 비극을 심화 시키고 있는 절대惡 이승준이 불러올 결말에 눈길을 뗄 수 없게 만든다.
이것은 밤에 나는 새들이 건물들에 부딫히도록 합니다.
대신 광장 민주주의에 기댄 포퓰리즘과 의사 전체주의가 똬리를 튼다.
당시 루루는 장나라와 장나라 아버지 주호성씨를 알게 되고 장나라가 한국에서 중문 에디션 음반 비상飞翔을 발매할 당시 그녀의 홍보활동을 도우면서 처음으로 한국과도 인연을 맺게 된다.
솅겐조약은 천구백구십오년 채택된 국경개방 조약으로 유럽연합 이십팔개 회원국 중 이십이개국과 비회원국인 노르웨이. 리히텐슈타인.스위스.아이슬란드를 포함해 총 모두 이십육개국이 가입돼 있다.
홓길동이구요 폰 번호는 백이십삼만 사천오백육십칠입니다
말레콘을 걷다 버스킹을 하는 사람들을 만났고 류준열은 그들에게 팁을 건넸으나 되레 더 많은 돈을 요구하기도 했다.
미쳴이 태어난 날이요
주로 동해와 남해에서만 나던 우렁쉥이를 경기 서해안 해역에서도 양식할 수 있는 가능성이 열렸다.
이사장은 이사 중에서 호선하며 재단을 대표하여 업무를 총괄하고 소속 직원을 지휘ㆍ감독한다.
ㅈ까라 그래.
세계랭킹 이위 더스틴 존슨과 잰더 쇼플리가 사타씩을 줄여 공동 이위에 올랐고 통산 오승 중 삼승을 메이저대회에서

모낰이 아파트 소유자에요?
그럼 웃는 거 한번 봅시다. 훌륭하신 닥터 얭께서 그 찌푸린 얼굴을
김병연 단장은 오천년 활의 정신문화를 품고 있는 세계적인 활의 도시 대구 북구의 역사적 가치를 널리 알리고자 죽궁에 전통 옻을 칠했다고 밝혔다.
정말 모든 아버지들ㅇ ...
알렉스틑 동쪽으로 향했어 알렉스가 보이는 사람 있ㅇ?
스캔을 시작해주세요 으어허ㅓㄺ!
이어 신 회장이 대통령이 면세점 정책에 영향을 미칠 수 있다는 점을 알고 그 직무 집행의 대가로 칠십억원을 건넸다고 판단했다.
그는 아내를 말태우는게 아니라 리안나에게 무릎 꿇고 장미를 건넸어
ö 그에게 말하지 않지.
가엾은 우리 딸
시장관리자는 시장에 설치된 주요 시설물과 편의시설에 대하여 전기ㆍ가스ㆍ소방 등 관련 전문 업체를 통해 정기적인 점검을 실시하는 등 화재예방 및 안전 확보를 위한 조치를 하여야 한다.
그 밖에 다양한 주제의 화장실 공간이 연출될 수 있도록 에어타올ㆍ그림ㆍ사진ㆍ화분 등을 설치할 수 있다.
ö 사랑 지금까지!
구청장은 제일항의 규정에 의한 기금운용계획서와 기금결산서를 매회계년도마다 각각 세입ㆍ세출예산안 또는 결산서와 함께 노원구의회에 제출하여야 한다.
보육교사는 노원구에 소재하는 국ㆍ공립 또는 민간보육시설에 종사하고 있는 자로 한다.
제십팔조 ① 구청장은 제십일조 제십이조 및 제십오조에 따른 음식물류 폐기물 배출방법 등에 따라 배출하지 아니한 자 또는 제십육조에 따른 준수사항을 지키지 아니한 자에 대하여 법 제육십팔조에 따라 과태료를 부과하여야 한다.
지난 칠일 영국 런던 웸블리 스타디움에서 펼쳐졌던 브라이튼과의 잉글랜드축구협회컵 사강전에서는 멘디가 나섰으나 완벽한 모습을 보여주지는 못했다.
국민일보와 국제사랑재단이 주최한 예배에서 오정현 목사는 그리스도인이 긍휼의 마음을 품고 고통당하는 북한 아이들에게 사랑의 본을 보여줬으면 좋겠다고 말했다.
영광은 네 몫이다 쨔샤
á ñ?
우리 좀 쉽게 해결하면 안됄까?
이것은 한국 드라마가 중국인들에게 대중적인 매력을 잃었기 때문이겠지만 다른 한편으

저기 부딫히지 않을까요?
이날 밤 동부동에 사는 머슴들이 횃불을 들고 마을 뒷산에 올라가 만세를 고창하였다.
실제 지난해 미국과의 무역전쟁이 시작된 후 실물경제가 급격하게 위축되면서 중국에서는 디돤런커우로 불리는 하층민의 불만이 급격히 커지는 분위기다.
얭이 저한테 지분을 남긴 데에는 이유가 있다고 생각해요
운동에 소질 있는 여성들이 축구를 앞다퉈 하는 등 인기와 투자에서 북한 최고의 스포츠로 자리매김하고 있다.


In [41]:
test = '()'
test_co = list()

for ch in test:
    if ch not in acoustic_labels:
        break
    test_co.append(test)
test_co

[]

In [40]:
len(final_corpus)

3001957

In [ ]:
char_label_df = pd.read_csv('train_labels.csv')

* 데이터셋 피클로 저장

In [39]:
import pickle

with open('kor_corpus.bin', 'wb') as f:
    pickle.dump(kor_corpus, f)